In [1]:
import anndata as ad
from pycombat import Combat
import numpy as np
import pandas as pd
import os

In [2]:
correspondence = {}
correspondence['149Sm_CREB'] ='149Sm_pCREB'
correspondence['167Er_ERK'] ='167Er_pERK12'
correspondence['164Dy_IkB'] ='164Dy_IkB'
correspondence['159Tb_MAPKAPK2'] ='159Tb_pMAPKAPK2'
correspondence['166Er_NFkB'] ='166Er_pNFkB'
correspondence['151Eu_p38'] ='151Eu_pp38'
correspondence['155Gd_S6'] ='155Gd_pS6'
correspondence['153Eu_STAT1'] ='153Eu_pSTAT1'
correspondence['154Sm_STAT3'] ='154Sm_pSTAT3'
correspondence['150Nd_STAT5'] ='150Nd_pSTAT5'
correspondence['168Er_pSTAT6'] ='168Yb_pSTAT6'
correspondence['174Yb_HLADR'] ='174Yb_HLADR'
correspondence['169Tm_CD25'] ='169Tm_CD25'

In [4]:
def batch_correction_combat(correspondence,perio_path,surge_path,stim):
    perio = ad.read_h5ad(perio_path)
    surge = ad.read_h5ad(surge_path)

    if stim=='LPS':
        perio_stim='P. gingivalis'
    else:
        perio_stim=stim
    # Load AnnData
    perio_unstim=perio[perio.obs['drug']=='Unstim']
    surge_unstim=surge[surge.obs['drug']=='Unstim']

    surge_stim=surge[surge.obs['drug']==stim]
    perio_stim=perio[perio.obs['drug']==perio_stim]

    # Choose the features
    features=correspondence.values()
    features = list(features)

    surge_unstim = surge_unstim[:, features]
    surge_stim = surge_stim[:, features]

    # Rename the features in perio_unstim to match those in surge_unstim
    perio_unstim.var_names = perio_unstim.var_names.to_series().replace(correspondence).values
    perio_stim.var_names = perio_stim.var_names.to_series().replace(correspondence).values
    perio_unstim = perio_unstim[:, features]
    perio_stim = perio_stim[:, features]

    assert perio_unstim.var_names.equals(surge_unstim.var_names)
    
    combined_unstim = ad.concat(
        {"perio": perio_unstim, "surge": surge_unstim},
        label="batch"
    )

    combat = Combat()
    X_corrected_unstim = combat.fit_transform(combined_unstim.X, combined_unstim.obs["batch"])

    surge_unstim_corrected = surge_unstim.copy()
    surge_unstim_corrected.X = X_corrected_unstim[combined_unstim.obs["batch"] == "surge"]

    perio_unstim_corrected = perio_unstim.copy()
    perio_unstim_corrected.X = X_corrected_unstim[combined_unstim.obs["batch"] == "perio"]
    
    # Create a dummy row for the perio batch
    dummy_surge = surge_stim[:1].copy()  # copy one row from surge_stim
    dummy_surge.X = np.zeros_like(dummy_surge.X)  # zero values
    dummy_surge.obs['batch'] = 'perio'

    # Concat dummy + real surge_stim
    combined_stim_surge = ad.concat(
        {"surge": surge_stim, "perio": dummy_surge},
        label="batch"
    )

    # Perform batch correction on the combined data
    X_surge_corrected = combat.transform(combined_stim_surge.X,combined_stim_surge.obs['batch'])  # Only transform, don't fit
    surge_stim_corrected = surge_stim.copy()
    surge_stim_corrected.X = X_surge_corrected[combined_stim_surge.obs["batch"] == "surge"]

    surge_corrected=ad.concat(
        {"Unstim": surge_unstim_corrected, stim: surge_stim_corrected},
        label="stim"
    )

    dummy_perio = perio_stim[:1].copy()  # copy one row from surge_stim
    dummy_perio.X = np.zeros_like(dummy_perio.X)  # zero values
    dummy_perio.obs['batch'] = 'surge'

    # Concat dummy + real surge_stim
    combined_stim_perio = ad.concat(
        {"surge": dummy_perio, "perio": perio_stim},
        label="batch"
    )

    # Perform batch correction on the combined data
    X_perio_corrected = combat.transform(combined_stim_perio.X,combined_stim_perio.obs['batch'])  # Only transform, don't fit
    perio_stim_corrected = perio_stim.copy()
    perio_stim_corrected.X = X_perio_corrected[combined_stim_perio.obs["batch"] == "perio"]

    perio_corrected=ad.concat(
        {"Unstim": perio_unstim_corrected, stim: perio_stim_corrected},
        label="stim"
    )

    surge_corrected = surge_corrected.copy()
    surge_corrected.strings_to_categoricals()

    for col in surge_corrected.obs.columns:
        if isinstance(surge_corrected.obs[col].dtype, pd.CategoricalDtype):
            surge_corrected.obs[col] = surge_corrected.obs[col].astype(str)
    surge_name= surge_path.split('/')[-1]
    surge_corrected.write('surge_train_on_perio_corrected_simple/'+surge_name)

    perio_corrected = perio_corrected.copy()
    perio_corrected.strings_to_categoricals()

    for col in perio_corrected.obs.columns:
        if isinstance(perio_corrected.obs[col].dtype, pd.CategoricalDtype):
            perio_corrected.obs[col] = perio_corrected.obs[col].astype(str)
    perio_name= perio_path.split('/')[-1]
    perio_corrected.write('perio_on_surge_train_corrected_simple/'+perio_name)

    return

In [7]:
directory_perio='perio_just_concat'
directory_surge='surge_just_concat_train'
listfile_perio=[f for f in os.listdir(directory_perio) if 'IFNa' not in f]
listfile_surge=[f for f in os.listdir(directory_surge) if 'IFNa' not in f]
listfile_perio.sort()
listfile_surge.sort()
for filename_perio,filename_surge in zip(listfile_perio,listfile_surge):
    path_perio=os.path.join(directory_perio, filename_perio)
    path_surge=os.path.join(directory_surge, filename_surge)
    if 'LPS' in filename_surge:
        batch_correction_combat(correspondence, path_perio, path_surge,'LPS')
    else:
        batch_correction_combat(correspondence, path_perio, path_surge,'TNFa')
    print(f"Corrected {filename_perio} and {filename_surge}")

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique,

Corrected perio_data_sherlock_P. gingivalis_B-Cells (CD19+CD3-).h5ad and surge_concatenated_LPS_B-Cells (CD19+CD3-).h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique,

Corrected perio_data_sherlock_P. gingivalis_CD4 T-Cells.h5ad and surge_concatenated_LPS_CD4 T-Cells.h5ad
Corrected perio_data_sherlock_P. gingivalis_CD56+CD16- NK Cells.h5ad and surge_concatenated_LPS_CD56+CD16- NK Cells.h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique,

...

Corrected perio_data_sherlock_TNFa_Non-classical Monocytes (CD14-CD16+).h5ad and surge_concatenated_TNFa_Non-classical Monocytes (CD14-CD16+).h5ad


Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
... storing 'stim' as categorical
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categorical
... storing 'stim' as categorical
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make 

Corrected perio_data_sherlock_TNFa_Tregs (CD25+FoxP3+).h5ad and surge_concatenated_TNFa_Tregs (CD25+FoxP3+).h5ad


... storing 'cell_type' as categorical
... storing 'patient' as categorical
... storing 'stim' as categorical
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not uniq

Corrected perio_data_sherlock_TNFa_mDCs (CD11c+HLADR+).h5ad and surge_concatenated_TNFa_mDCs (CD11c+HLADR+).h5ad
Corrected perio_data_sherlock_TNFa_pDCs(CD123+HLADR+).h5ad and surge_concatenated_TNFa_pDCs(CD123+HLADR+).h5ad


In [4]:
batch_correction_combat(correspondence, 'perio_data_sherlock2/perio_data_sherlockP. gingivalis_mDCs (CD11c+HLADR+)_train.h5ad', 'surge_concatenated/surge_concatenated_LPS_mDCs (CD11c+HLADR+).h5ad','LPS')

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
... storing 'drug' as categorical
... storing 'drug' as categorical
... storing 'cell_type' as categorical
... storing 'patient' as categori